# Домашнее задание №6. Нейронные сети
Суть этого задания в том, чтобы поиграться с параметрами сети, наблюдая за тем, как меняются результаты в зависимости от ваших действий.

## Задание 1
Скопируйте результаты первых трех заданий из Домашнего задания №5. У вас должен быть скачан и обработан датасет с данными о пассажирах Титаника. Получившийся DataFrame должен быть разделен на обучающую (80%) и тестовую выборку (20%).

In [1]:
# Напишите свой код в данной ячейке
import pandas as pd
from sklearn.model_selection import train_test_split

train_data = pd.read_csv("train.csv")

train_data['Sex'] = train_data['Sex'].map({'male': 0, 'female': 1})
train_data = train_data[['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']].dropna()

X = train_data.drop('Survived', axis=1)
y = train_data['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Код, рассмотренный на семинаре

In [2]:
import numpy as np

#### Инициализация

In [3]:
def init_layers(nn_architecture, seed = 99):
    np.random.seed(seed)
    number_of_layers = len(nn_architecture)
    params_values = {}

    for idx, layer in enumerate(nn_architecture):
        layer_idx = idx + 1

        layer_input_size = layer["input_dim"]
        layer_output_size = layer["output_dim"]

        params_values['W' + str(layer_idx)] = np.random.randn(
            layer_output_size, layer_input_size) * 0.1
        params_values['b' + str(layer_idx)] = np.random.randn(
            layer_output_size, 1) * 0.1

    return params_values

#### Функции активации

In [4]:
def sigmoid(Z):
    return 1/(1+np.exp(-Z))

def relu(Z):
    return np.maximum(0,Z)

def sigmoid_backward(dA, Z):
    sig = sigmoid(Z)
    return dA * sig * (1 - sig)

def relu_backward(dA, Z):
    dZ = np.array(dA, copy = True)
    dZ[Z <= 0] = 0
    return dZ

#### Прямой проход

In [5]:
def single_layer_forward_propagation(A_prev, W_curr, b_curr, activation="relu"):
    Z_curr = np.dot(W_curr, A_prev) + b_curr

    if activation == "relu":
        activation_func = relu
    elif activation == "sigmoid":
        activation_func = sigmoid
    else:
        raise Exception('Non-supported activation function')

    return activation_func(Z_curr), Z_curr

def full_forward_propagation(X, params_values, nn_architecture):
    memory = {}
    A_curr = X

    for idx, layer in enumerate(nn_architecture):
        layer_idx = idx + 1
        A_prev = A_curr

        activ_function_curr = layer["activation"]
        W_curr = params_values["W" + str(layer_idx)]
        b_curr = params_values["b" + str(layer_idx)]
        A_curr, Z_curr = single_layer_forward_propagation(A_prev, W_curr, b_curr, activ_function_curr)

        memory["A" + str(idx)] = A_prev
        memory["Z" + str(layer_idx)] = Z_curr

    return A_curr, memory

#### Вычисление ошибки

In [6]:
def get_cost_value(Y_hat, Y):
    m = Y_hat.shape[1]
    cost = -1 / m * (np.dot(Y, np.log(Y_hat).T) + np.dot(1 - Y, np.log(1 - Y_hat).T))
    return np.squeeze(cost)

#### Обратный проход

In [7]:
def single_layer_backward_propagation(dA_curr, W_curr, b_curr, Z_curr, A_prev, activation="relu"):
    m = A_prev.shape[1]

    if activation == "relu":
        backward_activation_func = relu_backward
    elif activation == "sigmoid":
        backward_activation_func = sigmoid_backward
    else:
        raise Exception('Non-supported activation function')

    dZ_curr = backward_activation_func(dA_curr, Z_curr)

    dW_curr = np.dot(dZ_curr, A_prev.T) / m
    db_curr = np.sum(dZ_curr, axis=1, keepdims=True) / m
    dA_prev = np.dot(W_curr.T, dZ_curr)

    return dA_prev, dW_curr, db_curr

def full_backward_propagation(Y_hat, Y, memory, params_values, nn_architecture):
    grads_values = {}

    m = Y.shape[1]
    Y = Y.reshape(Y_hat.shape)

    dA_prev = - (np.divide(Y, Y_hat) - np.divide(1 - Y, 1 - Y_hat));

    for layer_idx_prev, layer in reversed(list(enumerate(nn_architecture))):
        layer_idx_curr = layer_idx_prev + 1
        activ_function_curr = layer["activation"]

        dA_curr = dA_prev

        A_prev = memory["A" + str(layer_idx_prev)]
        Z_curr = memory["Z" + str(layer_idx_curr)]

        W_curr = params_values["W" + str(layer_idx_curr)]
        b_curr = params_values["b" + str(layer_idx_curr)]

        dA_prev, dW_curr, db_curr = single_layer_backward_propagation(
            dA_curr, W_curr, b_curr, Z_curr, A_prev, activ_function_curr)

        grads_values["dW" + str(layer_idx_curr)] = dW_curr
        grads_values["db" + str(layer_idx_curr)] = db_curr

    return grads_values

#### Обновление весов

In [8]:
def update(params_values, grads_values, nn_architecture, learning_rate):

    for layer_idx, layer in enumerate(nn_architecture, 1):
        params_values["W" + str(layer_idx)] -= learning_rate * grads_values["dW" + str(layer_idx)]
        params_values["b" + str(layer_idx)] -= learning_rate * grads_values["db" + str(layer_idx)]

    return params_values

#### Оценка результата

In [9]:
def convert_prob_into_class(probs):
    probs_ = np.copy(probs)
    probs_[probs_ > 0.5] = 1
    probs_[probs_ <= 0.5] = 0
    return probs_

def get_accuracy_value(Y_hat, Y):
    Y_hat_ = convert_prob_into_class(Y_hat)
    return (Y_hat_ == Y).all(axis=0).mean()

#### Функция для обучения

In [10]:
def train(X, Y, nn_architecture, epochs, learning_rate, verbose=False, callback=None):
    params_values = init_layers(nn_architecture, 2)
    cost_history = []
    accuracy_history = []

    for i in range(epochs):
        Y_hat, cashe = full_forward_propagation(X, params_values, nn_architecture)

        cost = get_cost_value(Y_hat, Y)
        cost_history.append(cost)
        accuracy = get_accuracy_value(Y_hat, Y)
        accuracy_history.append(accuracy)

        grads_values = full_backward_propagation(Y_hat, Y, cashe, params_values, nn_architecture)
        params_values = update(params_values, grads_values, nn_architecture, learning_rate)

        if(i % 50 == 0):
            if(verbose):
                print("Iteration: {:05} - cost: {:.5f} - accuracy: {:.5f}".format(i, cost, accuracy))
            if(callback is not None):
                callback(i, params_values)

    return params_values

#### Ячейка с параметрами слоев

In [11]:
#NN_ARCHITECTURE = [
#    {"input_dim": 2, "output_dim": 25, "activation": "relu"},
#    {"input_dim": 25, "output_dim": 50, "activation": "relu"},
#    {"input_dim": 50, "output_dim": 50, "activation": "relu"},
#    {"input_dim": 50, "output_dim": 25, "activation": "relu"},
#    {"input_dim": 25, "output_dim": 1, "activation": "sigmoid"},
#]

#### Ячейка с обучением

In [12]:
# Обучение
#params_values = train(np.transpose(X_train), np.transpose(y_train.reshape((y_train.shape[0], 1))), NN_ARCHITECTURE, 100, 0.01)
# Предсказание
#Y_test_hat, _ = full_forward_propagation(np.transpose(X_test), params_values, NN_ARCHITECTURE)
# Оценка на тестовой выборке
#acc_test = get_accuracy_value(Y_test_hat, np.transpose(y_test.reshape((y_test.shape[0], 1))))
#print("Test set accuracy: {:.2f}".format(acc_test))

## Задание 2. Запуск на данных
Модифицируйте код в последних двух ячейках так, чтобы его можно было запускать на подготовленных данных. Запишите исправленные ячейки ниже и запустите.

In [13]:
# Исправленная ячейка с параметрами слоев
NN_ARCHITECTURE = [
    {"input_dim": 7, "output_dim": 25, "activation": "relu"},
    {"input_dim": 25, "output_dim": 50, "activation": "relu"},
    {"input_dim": 50, "output_dim": 50, "activation": "relu"},
    {"input_dim": 50, "output_dim": 25, "activation": "relu"},
    {"input_dim": 25, "output_dim": 1, "activation": "sigmoid"},
]

In [14]:
# Исправленная ячейка с обучением
params_values = train(np.transpose(X_train), np.transpose(y_train.values.reshape((y_train.shape[0], 1))), NN_ARCHITECTURE, 100, 0.01)   # Обучение
Y_test_hat, _ = full_forward_propagation(np.transpose(X_test), params_values, NN_ARCHITECTURE)                                          # Предсказание
acc_test = get_accuracy_value(Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1))))                                    # Оценка на тестовой выборке
print("Test set accuracy: {:.2f}".format(acc_test))

Test set accuracy: 0.63


## Задание 3. Нормализация данных
На результат обучения влияет то, что данные в разных столбцах могут принимать совсем разные по величине значения, все измеряются в разных единицах. Нормализуйте данные, например, при помощи <code>sklearn.preprocessing.StandardScaler</code>. А затем снова скопируйте сюда ячейку с обучением и запустите, чтобы оценить, как меняется результат.

In [15]:
# Напишите свой код в данной ячейке
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


# Исправленная ячейка с обучением
params_values = train(np.transpose(X_train), np.transpose(y_train.values.reshape((y_train.shape[0], 1))), NN_ARCHITECTURE, 100, 0.01)   # Обучение
Y_test_hat, _ = full_forward_propagation(np.transpose(X_test), params_values, NN_ARCHITECTURE)                                          # Предсказание
acc_test = get_accuracy_value(Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1))))                                    # Оценка на тестовой выборке
print("Test set accuracy: {:.2f}".format(acc_test))

Test set accuracy: 0.61


## Задание 4. Число эпох
Попробуйте обучать, используя разные значения числа эпох.

**10 эпох**

In [16]:
# Ячейка с обучением, в которой число эпох равно 10
X = train_data.drop('Survived', axis=1)
y = train_data['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Исправленная ячейка с обучением (30 эпох)
params_values = train(np.transpose(X_train), np.transpose(y_train.values.reshape((y_train.shape[0], 1))), NN_ARCHITECTURE, 10, 0.01)    # Обучение
Y_test_hat, _ = full_forward_propagation(np.transpose(X_test), params_values, NN_ARCHITECTURE)                                          # Предсказание
acc_test = get_accuracy_value(Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1))))                                    # Оценка на тестовой выборке
print("Test set accuracy: {:.2f}".format(acc_test))


Test set accuracy: 0.59


**1000 эпох**

In [17]:
# Исправленная ячейка с обучением (1000 эпох)
params_values = train(np.transpose(X_train), np.transpose(y_train.values.reshape((y_train.shape[0], 1))), NN_ARCHITECTURE, 1000, 0.01)   # Обучение
Y_test_hat, _ = full_forward_propagation(np.transpose(X_test), params_values, NN_ARCHITECTURE)                                          # Предсказание
acc_test = get_accuracy_value(Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1))))                                    # Оценка на тестовой выборке
print("Test set accuracy: {:.2f}".format(acc_test))

Test set accuracy: 0.66


**10'000 эпох** 

In [18]:
# Исправленная ячейка с обучением (10000 эпох)
params_values = train(np.transpose(X_train), np.transpose(y_train.values.reshape((y_train.shape[0], 1))), NN_ARCHITECTURE, 10000, 0.01)   # Обучение
Y_test_hat, _ = full_forward_propagation(np.transpose(X_test), params_values, NN_ARCHITECTURE)                                          # Предсказание
acc_test = get_accuracy_value(Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1))))                                    # Оценка на тестовой выборке
print("Test set accuracy: {:.2f}".format(acc_test))

Test set accuracy: 0.64


## Задание 5. Скорость обучения
Попробуйте обучать, используя разные значения скорости обучения. Число эпох выберите на выше усмотрение, но одинаковое во всех трех ячейках.

**0,0001**

In [19]:
# Исправленная ячейка с обучением (10000 эпох)
params_values = train(np.transpose(X_train), np.transpose(y_train.values.reshape((y_train.shape[0], 1))), NN_ARCHITECTURE, 100, 0.0001)   # Обучение
Y_test_hat, _ = full_forward_propagation(np.transpose(X_test), params_values, NN_ARCHITECTURE)                                          # Предсказание
acc_test = get_accuracy_value(Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1))))                                    # Оценка на тестовой выборке
print("Test set accuracy: {:.2f}".format(acc_test))

Test set accuracy: 0.61


**0,001**

In [20]:
# Исправленная ячейка с обучением (10000 эпох)
params_values = train(np.transpose(X_train), np.transpose(y_train.values.reshape((y_train.shape[0], 1))), NN_ARCHITECTURE, 100, 0.001)   # Обучение
Y_test_hat, _ = full_forward_propagation(np.transpose(X_test), params_values, NN_ARCHITECTURE)                                          # Предсказание
acc_test = get_accuracy_value(Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1))))                                    # Оценка на тестовой выборке
print("Test set accuracy: {:.2f}".format(acc_test))

Test set accuracy: 0.60


**0,01**

In [21]:
# Исправленная ячейка с обучением (10000 эпох)
params_values = train(np.transpose(X_train), np.transpose(y_train.values.reshape((y_train.shape[0], 1))), NN_ARCHITECTURE, 100, 0.01)   # Обучение
Y_test_hat, _ = full_forward_propagation(np.transpose(X_test), params_values, NN_ARCHITECTURE)                                          # Предсказание
acc_test = get_accuracy_value(Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1))))                                    # Оценка на тестовой выборке
print("Test set accuracy: {:.2f}".format(acc_test))

Test set accuracy: 0.63


## Задание 6. Изменение слоев
Начиная с этого задания вы можете выбирать число эпох и скорость обучения самостоятельно.

А еще теперь кроме ячейки с обучением будет меняться еще и ячейка с параметрами слоев. То есть каждая ячейка с ответом будет выглядеть как-то так:

In [22]:
# Какой-то комментарий про эту ячейку
#NN_ARCHITECTURE = [
#    {"input_dim": 2, "output_dim": 25, "activation": "relu"},
#    {"input_dim": 25, "output_dim": 50, "activation": "relu"},
#    {"input_dim": 50, "output_dim": 50, "activation": "relu"},
#    {"input_dim": 50, "output_dim": 25, "activation": "relu"},
#    {"input_dim": 25, "output_dim": 1, "activation": "sigmoid"},
#]
# Обучение
#params_values = train(np.transpose(X_train), np.transpose(y_train.reshape((y_train.shape[0], 1))), NN_ARCHITECTURE, 10000, 0.01)
# Предсказание
#Y_test_hat, _ = full_forward_propagation(np.transpose(X_test), params_values, NN_ARCHITECTURE)
# Оценка на тестовой выборке
#acc_test = get_accuracy_value(Y_test_hat, np.transpose(y_test.reshape((y_test.shape[0], 1))))
#print("Test set accuracy: {:.2f}".format(acc_test))

После каждого изменения параметров, не забывайте вставлять код с обучением и выводом accuracy.

In [23]:
# Ячейка с обучением и параметрами слоев, в которой все функции активации это sigmoid
NN_ARCHITECTURE = [
    {"input_dim": 7, "output_dim": 25, "activation": "sigmoid"},
    {"input_dim": 25, "output_dim": 50, "activation": "sigmoid"},
    {"input_dim": 50, "output_dim": 50, "activation": "sigmoid"},
    {"input_dim": 50, "output_dim": 25, "activation": "sigmoid"},
    {"input_dim": 25, "output_dim": 1, "activation": "sigmoid"},
]

# Исправленная ячейка с обучением (10000 эпох)
params_values = train(np.transpose(X_train), np.transpose(y_train.values.reshape((y_train.shape[0], 1))), NN_ARCHITECTURE, 100, 0.01)   # Обучение
Y_test_hat, _ = full_forward_propagation(np.transpose(X_test), params_values, NN_ARCHITECTURE)                                          # Предсказание
acc_test = get_accuracy_value(Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1))))                                    # Оценка на тестовой выборке
print("Test set accuracy: {:.2f}".format(acc_test))

Test set accuracy: 0.61


In [24]:
# Ячейка с обучением и параметрами слоев, в которой есть только последний выходной слой с sigmoid
NN_ARCHITECTURE = [
    {"input_dim": 7, "output_dim": 25, "activation": "sigmoid"},
    {"input_dim": 25, "output_dim": 1, "activation": "sigmoid"},
]

# Исправленная ячейка с обучением (10000 эпох)
params_values = train(np.transpose(X_train), np.transpose(y_train.values.reshape((y_train.shape[0], 1))), NN_ARCHITECTURE, 100, 0.01)   # Обучение
Y_test_hat, _ = full_forward_propagation(np.transpose(X_test), params_values, NN_ARCHITECTURE)                                          # Предсказание
acc_test = get_accuracy_value(Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1))))                                    # Оценка на тестовой выборке
print("Test set accuracy: {:.2f}".format(acc_test))

Test set accuracy: 0.61


In [29]:
# Ячейка с обучением и параметрами слоев, в которой есть два слоя с relu и выходной слой с sigmoid
NN_ARCHITECTURE = [
    {"input_dim": 7, "output_dim": 25, "activation": "relu"},
    {"input_dim": 25, "output_dim": 50, "activation": "sigmoid"},
    {"input_dim": 50, "output_dim": 50, "activation": "relu"},
    {"input_dim": 50, "output_dim": 25, "activation": "sigmoid"},
    {"input_dim": 25, "output_dim": 1, "activation": "sigmoid"},
]


# Исправленная ячейка с обучением (10000 эпох)
params_values = train(np.transpose(X_train), np.transpose(y_train.values.reshape((y_train.shape[0], 1))), NN_ARCHITECTURE, 100, 0.01)   # Обучение
Y_test_hat, _ = full_forward_propagation(np.transpose(X_test), params_values, NN_ARCHITECTURE)                                          # Предсказание
acc_test = get_accuracy_value(Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1))))                                    # Оценка на тестовой выборке
print("Test set accuracy: {:.2f}".format(acc_test))

Test set accuracy: 0.61


In [26]:
# Ячейка с обучением и параметрами слоев, в которой есть два слоя с relu и выходной слой с sigmoid
# а еще число нейронов на каждом слое равно 8 (кроме входного и выходного)
NN_ARCHITECTURE = [
    {"input_dim": 7, "output_dim": 8, "activation": "sigmoid"},
    {"input_dim": 8, "output_dim": 8, "activation": "relu"},
    {"input_dim": 8, "output_dim": 8, "activation": "relu"},
    {"input_dim": 8, "output_dim": 8, "activation": "sigmoid"},
    {"input_dim": 8, "output_dim": 1, "activation": "sigmoid"},
]


# Исправленная ячейка с обучением (10000 эпох)
params_values = train(np.transpose(X_train), np.transpose(y_train.values.reshape((y_train.shape[0], 1))), NN_ARCHITECTURE, 100, 0.01)   # Обучение
Y_test_hat, _ = full_forward_propagation(np.transpose(X_test), params_values, NN_ARCHITECTURE)                                          # Предсказание
acc_test = get_accuracy_value(Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1))))                                    # Оценка на тестовой выборке
print("Test set accuracy: {:.2f}".format(acc_test))

Test set accuracy: 0.61


In [27]:
# Ячейка с обучением и параметрами слоев, в которой есть два слоя с relu и выходной слой с sigmoid
# а еще число нейронов на каждом слое равно 100 (кроме входного и выходного)
NN_ARCHITECTURE = [
    {"input_dim": 7, "output_dim": 100, "activation": "sigmoid"},
    {"input_dim": 100, "output_dim": 100, "activation": "sigmoid"},
    {"input_dim": 100, "output_dim": 100, "activation": "relu"},
    {"input_dim": 100, "output_dim": 100, "activation": "relu"},
    {"input_dim": 100, "output_dim": 1, "activation": "sigmoid"},
]

# Исправленная ячейка с обучением (10000 эпох)
params_values = train(np.transpose(X_train), np.transpose(y_train.values.reshape((y_train.shape[0], 1))), NN_ARCHITECTURE, 100, 0.01)   # Обучение
Y_test_hat, _ = full_forward_propagation(np.transpose(X_test), params_values, NN_ARCHITECTURE)                                          # Предсказание
acc_test = get_accuracy_value(Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1))))                                    # Оценка на тестовой выборке
print("Test set accuracy: {:.2f}".format(acc_test))

Test set accuracy: 0.61


## Задание 7. Превзойти себя
Наконец, в этом задании вам предоставляется полная свобода действий в рамках этой модели. Используйте все полученные знания. Пробуйте другие модификации. Добейтесь результата как можно лучше.

In [28]:
# Ячейка с обучением и полной свободой действий
NN_ARCHITECTURE = [
    {"input_dim": 7, "output_dim": 50, "activation": "relu"},
    {"input_dim": 50, "output_dim": 50, "activation": "relu"},
    {"input_dim": 50, "output_dim": 1, "activation": "sigmoid"},
]

X = train_data.drop('Survived', axis=1)
y = train_data['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = preprocessing.StandardScaler().fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

params_values = train(np.transpose(X_train), np.transpose(y_train.values.reshape((y_train.shape[0], 1))), NN_ARCHITECTURE, 10000, 0.01)
Y_test_hat, _ = full_forward_propagation(np.transpose(X_test), params_values, NN_ARCHITECTURE)

# Оценка на тестовой выборке
acc_test = get_accuracy_value(Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1))))
print("Test set accuracy: {:.2f}".format(acc_test))

Test set accuracy: 0.82
